In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import itertools
from sklearn.model_selection import train_test_split, GridSearchCV, RepeatedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder, RobustScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.impute import SimpleImputer
from xgboost import XGBRegressor

In [32]:
df_modelo = pd.read_csv('df_EDA_predicprecio.csv')
df_modelo.head()

,make,model,version,fuel,year,kms,power,shift,price
0,Opel,Crossland,1.2 GAS 110 GS Line 5p S/S,Gasolina,2022,5.0,110.0,manual,22900
1,Opel,Crossland,1.2 81kW (110CV) GS Line,Gasolina,2022,24847.0,110.0,manual,19990
2,Opel,Crossland,1.5D 88kW (120CV) Business Elegance Auto,Diésel,2021,41356.0,120.0,automatic,18590
3,Opel,Crossland,GS-Line 1.2 GAS MT6 S/S 110cv,Gasolina,2022,11.0,110.0,manual,22700
4,Opel,Crossland,1.2 GS LINE 110 CV 5P,Gasolina,2021,51390.0,110.0,manual,18200


In [33]:
# Función de normalización ajustada
def normalize_version(row):
    version = row['version']
    year = row['year']
    
    # Paso 1: Convertir todo a minúsculas
    version = version.lower()
    
    # Paso 2: Eliminar paréntesis y unidades de potencia como CV, kW
    version = re.sub(r'\s*\(?\d+\s*(cv|kw)\)?\s*', '', version)
    
    # Paso 3: Normalizar palabras clave de combustible
    version = re.sub(r'\bgas\b', 'gasolina', version)
    version = re.sub(r'\bdiesel\b', 'diésel', version)
    
    # Paso 4: Eliminar abreviaturas de transmisión y tipo (MT6, S/S, 5p, auto, manual, etc.)
    version = re.sub(r'\b(mt6|s/s|5p|automatico|manual|auto|edition|line|style|pro|exclusive|gs-line)\b', '', version)
    
    # Paso 5: Eliminar texto que no aporta a la versión, como "cv" o "kw"
    version = re.sub(r'\b(cv|kw)\b', '', version)
    
    # Paso 6: Eliminar caracteres de puntuación innecesarios y espacios adicionales
    version = re.sub(r'\s+', ' ', version)  # Reemplazar múltiples espacios por uno solo
    version = re.sub(r'[^\w\s]', '', version)  # Eliminar caracteres no alfanuméricos (por ejemplo, guiones, paréntesis)
    
    # Paso 7: Eliminar espacios al principio y al final
    version = version.strip()
    
    # Incluir el año en la versión normalizada
    normalized_version = f"{version} {year}"
    
    return normalized_version

# Aplicar la normalización incluyendo el año directamente en df_modelo
df_modelo['normalized_version'] = df_modelo.apply(normalize_version, axis=1)

In [34]:
# Número de filas antes de eliminar duplicados
initial_rows = len(df_modelo)

# Eliminar duplicados basados en 'make', 'model' y 'normalized_version'
df_modelo = df_modelo.drop_duplicates(subset=['make', 'model', 'normalized_version'])

# Número de filas después de eliminar duplicados
final_rows = len(df_modelo)

# Número de filas eliminadas
deleted_rows = initial_rows - final_rows

print(f"\nNúmero de filas originales: {initial_rows}")
print(f"Número de filas después de eliminar duplicados: {final_rows}")
print(f"Número de filas eliminadas: {deleted_rows}")


Número de filas originales: 12453
Número de filas después de eliminar duplicados: 9010
Número de filas eliminadas: 3443


In [35]:
# Función de depuración de valores infinitos
def depurar_valores_infinitos(X):
    """
    Depura valores infinitos separando columnas numéricas y categóricas
    
    Parámetros:
    X (DataFrame): DataFrame de entrada
    
    Retorna:
    DataFrame procesado
    """
    # Separar columnas numéricas y categóricas
    columnas_numericas = X.select_dtypes(include=['int64', 'float64']).columns
    columnas_categoricas = X.select_dtypes(include=['object']).columns
    
    # Clonar el DataFrame
    X_depurado = X.copy()
    
    # Imputar columnas numéricas
    if len(columnas_numericas) > 0:
        # Convertir infinitos a NaN
        X_depurado[columnas_numericas] = X_depurado[columnas_numericas].replace([np.inf, -np.inf], np.nan)
        
        # Imputar valores NaN con la mediana
        imputer_numerico = SimpleImputer(strategy='median')
        X_depurado[columnas_numericas] = imputer_numerico.fit_transform(X_depurado[columnas_numericas])
    
    # Imputar columnas categóricas con el valor más frecuente
    if len(columnas_categoricas) > 0:
        imputer_categorico = SimpleImputer(strategy='most_frequent')
        X_depurado[columnas_categoricas] = imputer_categorico.fit_transform(X_depurado[columnas_categoricas])
    
    return X_depurado

In [36]:
# Función de creación segura de características
def crear_caracteristicas_seguras(df):
    """
    Crea características nuevas manejando casos especiales
    
    Parámetros:
    df (DataFrame): DataFrame original
    
    Retorna:
    DataFrame con nuevas características
    """
    # Clonar el dataframe para no modificar el original
    df_seguro = df.copy()
    
    # Manejar casos de car_age = 0
    df_seguro['car_age'] = 2024 - df_seguro['year']
    df_seguro.loc[df_seguro['car_age'] == 0, 'car_age'] = 1  # Evitar división por cero
    
    # Calcular kms_per_year con manejo de casos especiales
    df_seguro['kms_per_year'] = df_seguro['kms'] / df_seguro['car_age']
    
    # Manejar casos extremos de kms_per_year
    kms_per_year_mean = df_seguro['kms_per_year'].median()
    df_seguro.loc[np.isinf(df_seguro['kms_per_year']), 'kms_per_year'] = kms_per_year_mean
    
    # Calcular power_per_km con manejo de casos especiales
    df_seguro['power_per_km'] = df_seguro['power'] / df_seguro['kms']
    power_per_km_mean = df_seguro['power_per_km'].median()
    df_seguro.loc[np.isinf(df_seguro['power_per_km']), 'power_per_km'] = power_per_km_mean
    
    return df_seguro


In [37]:
# Paso 1: Aplicar función de características
df_modelo = crear_caracteristicas_seguras(df_modelo)

In [38]:
# Paso 2: Depuración inicial de todo el dataset
df_modelo = depurar_valores_infinitos(df_modelo)

In [39]:
# Paso 3: Identificar columnas
columnas_categoricas = ['make', 'model', 'version', 'fuel', 'shift']
columnas_numericas = ['year', 'kms', 'power', 'car_age', 'kms_per_year', 'power_per_km']

In [40]:
# Paso 6: Preparación de datos
# Separar características y objetivo
X = df_modelo.drop('price', axis=1)
y = df_modelo['price']

In [41]:
# Dividir datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [42]:
# Paso 7: Preprocesador robusto
preprocessor = ColumnTransformer(
    transformers=[
        ('num', RobustScaler(), columnas_numericas),
        ('cat', OneHotEncoder(handle_unknown='ignore'), columnas_categoricas)
    ])

In [43]:
# Inicializar un diccionario para guardar los resultados
resultados_modelos = {}

# Depurar valores infinitos una vez
X_train_depurado = depurar_valores_infinitos(X_train)
X_test_depurado = depurar_valores_infinitos(X_test)

In [44]:
# === Celda para RandomForest ===

# Definir modelo y parámetros
nombre = 'RandomForest'
modelo = RandomForestRegressor(random_state=42)
parametros = {
    'n_estimators': [100, 200, 300, 400],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10, 15],
    'max_features': ['sqrt', 'log2', None],  # Corregido: 'auto' por 'sqrt', None
    'bootstrap': [True, False]
}

# Crear pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', modelo)
])

# Grid Search con validación cruzada repetida
grid_search = GridSearchCV(
    pipeline, 
    param_grid={f'regressor__{k}': v for k, v in parametros.items()}, 
    cv=RepeatedKFold(n_splits=5, n_repeats=3),
    scoring=['neg_mean_squared_error', 'r2'],  # Usar múltiples métricas
    refit='r2',  # Reajustar usando R2
    verbose=1,   # Mostrar progreso
    n_jobs=-1    # Usar todos los núcleos
)

In [ ]:
# Entrenar el modelo
print(f"Iniciando entrenamiento de {nombre}...")
grid_search.fit(X_train_depurado, y_train)


Iniciando entrenamiento de RandomForest...
Fitting 15 folds for each of 384 candidates, totalling 5760 fits


In [ ]:
# Obtener mejores parámetros
mejor_combinacion = grid_search.best_params_
print(f"\nMejores parámetros para {nombre}:")
for param, valor in mejor_combinacion.items():
    print(f"{param.replace('regressor__', '')}: {valor}")

# Evaluar en el conjunto de prueba
y_pred = grid_search.predict(X_test_depurado)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"\nResultados en conjunto de prueba para {nombre}:")
print(f"MSE: {mse:.4f}")
print(f"R²: {r2:.4f}")

In [ ]:
# Guardar resultados
resultados_modelos[nombre] = {
    'pipeline': grid_search,
    'mejor_configuracion': mejor_combinacion,
    'cv_results': grid_search.cv_results_,
    'test_mse': mse,
    'test_r2': r2
}

In [ ]:
# === Celda para Gradient Boosting ===

# Definir modelo y parámetros
nombre = 'GradientBoosting'
modelo = GradientBoostingRegressor(random_state=42)
parametros = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 4, 5],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'subsample': [0.8, 0.9, 1.0]
}

# Crear pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', modelo)
])

# Grid Search con validación cruzada repetida
grid_search = GridSearchCV(
    pipeline, 
    param_grid={f'regressor__{k}': v for k, v in parametros.items()}, 
    cv=RepeatedKFold(n_splits=5, n_repeats=3),
    scoring=['neg_mean_squared_error', 'r2'],  # Usar múltiples métricas
    refit='r2',  # Reajustar usando R2
    verbose=1,   # Mostrar progreso
    n_jobs=-1    # Usar todos los núcleos
)

In [ ]:
# Entrenar el modelo
print(f"Iniciando entrenamiento de {nombre}...")
grid_search.fit(X_train_depurado, y_train)


In [ ]:
# Obtener mejores parámetros
mejor_combinacion = grid_search.best_params_
print(f"\nMejores parámetros para {nombre}:")
for param, valor in mejor_combinacion.items():
    print(f"{param.replace('regressor__', '')}: {valor}")

# Evaluar en el conjunto de prueba
y_pred = grid_search.predict(X_test_depurado)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"\nResultados en conjunto de prueba para {nombre}:")
print(f"MSE: {mse:.4f}")
print(f"R²: {r2:.4f}")

In [ ]:
# Guardar resultados
resultados_modelos[nombre] = {
    'pipeline': grid_search,
    'mejor_configuracion': mejor_combinacion,
    'cv_results': grid_search.cv_results_,
    'test_mse': mse,
    'test_r2': r2
}

In [ ]:
# === Celda para XGBoost ===

# Definir modelo y parámetros
nombre = 'XGBoost'
modelo = XGBRegressor(random_state=42)
parametros = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 4, 5],
    'min_child_weight': [1, 3, 5],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0]
}

# Crear pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', modelo)
])

# Grid Search con validación cruzada repetida
grid_search = GridSearchCV(
    pipeline, 
    param_grid={f'regressor__{k}': v for k, v in parametros.items()}, 
    cv=RepeatedKFold(n_splits=5, n_repeats=3),
    scoring=['neg_mean_squared_error', 'r2'],  # Usar múltiples métricas
    refit='r2',  # Reajustar usando R2
    verbose=1,   # Mostrar progreso
    n_jobs=-1    # Usar todos los núcleos
)

In [ ]:
# Entrenar el modelo
print(f"Iniciando entrenamiento de {nombre}...")
grid_search.fit(X_train_depurado, y_train)

In [ ]:
# Obtener mejores parámetros
mejor_combinacion = grid_search.best_params_
print(f"\nMejores parámetros para {nombre}:")
for param, valor in mejor_combinacion.items():
    print(f"{param.replace('regressor__', '')}: {valor}")

# Evaluar en el conjunto de prueba
y_pred = grid_search.predict(X_test_depurado)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"\nResultados en conjunto de prueba para {nombre}:")
print(f"MSE: {mse:.4f}")
print(f"R²: {r2:.4f}")

In [ ]:
# Guardar resultados
resultados_modelos[nombre] = {
    'pipeline': grid_search,
    'mejor_configuracion': mejor_combinacion,
    'cv_results': grid_search.cv_results_,
    'test_mse': mse,
    'test_r2': r2
}

In [ ]:
# === Celda para comparar resultados ===

# Mostrar resumen de resultados para todos los modelos entrenados
if resultados_modelos:
    print("RESUMEN DE RESULTADOS:\n")
    
    # Crear tabla comparativa
    resultados_df = pd.DataFrame({
        'Modelo': [],
        'MSE Test': [],
        'R² Test': []
    })
    
    for nombre, resultado in resultados_modelos.items():
        nueva_fila = pd.DataFrame({
            'Modelo': [nombre],
            'MSE Test': [resultado['test_mse']],
            'R² Test': [resultado['test_r2']]
        })
        resultados_df = pd.concat([resultados_df, nueva_fila], ignore_index=True)
    
    # Ordenar por R² (descendente)
    resultados_df = resultados_df.sort_values('R² Test', ascending=False).reset_index(drop=True)
    
    # Mostrar tabla
    print(resultados_df)
    
    # Identificar el mejor modelo
    mejor_modelo = resultados_df.iloc[0]['Modelo']
    mejor_r2 = resultados_df.iloc[0]['R² Test']
    mejor_mse = resultados_df.iloc[0]['MSE Test']
    
    print(f"\nMEJOR MODELO: {mejor_modelo}")
    print(f"R² Test: {mejor_r2:.4f}")
    print(f"MSE Test: {mejor_mse:.4f}")
    
    # Mostrar los mejores parámetros del mejor modelo
    print("\nMejores parámetros:")
    for param, valor in resultados_modelos[mejor_modelo]['mejor_configuracion'].items():
        print(f"{param.replace('regressor__', '')}: {valor}")
else:
    print("No hay modelos entrenados para comparar.")